In [1]:
'''
Gets text content for tweet IDs
'''

# standard
from __future__ import print_function
import getopt
import logging
import os
import sys
# import traceback
# third-party: `pip install tweepy`
import tweepy

import pandas as pd
from tqdm import tqdm

import seaborn as sns
%matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

# global logger level is configured in main()
Logger = None

In [2]:
# batch size depends on Twitter limit, 100 at this time
batch_size=100

# Generate your own at https://apps.twitter.com/app
CONSUMER_KEY = 'FTKEJzQ4N85xCqlLEgxKbQnxV'# 'Consumer Key (API key)'
CONSUMER_SECRET = 'eQPjDnxxOZQMNyPDTQSvjinKrmOuS2nQs0DnyLVzeLQJ96yCdb' # 'Consumer Secret (API Secret)'
OAUTH_TOKEN = '636585381-69wgslGdJgEjEH9Tw0PXZd3TzPVSogx49LdY2DP2' #'Access Token'
OAUTH_TOKEN_SECRET = 'TsG6hHLKjVhVnAYUm5JeWhr4wIyvhlrO6HFUNyfcuvyJZ' # 'Access Token Secret'

In [ ]:
# connect to twitter
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

def get_tweet_list(twapi, idlist):
    '''
    Invokes bulk lookup method.
    Raises an exception if rate limit is exceeded.
    '''
    
    
    # fetch as little metadata as possible
    tweets = []
    chunked_ids = list(chunks_ids(idlist, batch_size))
    
    for ids in chunked_ids: 
#         print(ids)
        tweets.append(twapi.statuses_lookup(id_=ids, include_entities=False, trim_user=True))
#     if len(idlist) != len(tweets):
#         print('get_tweet_list: unexpected response size %d, expected %d', len(tweets), len(idlist))
#     for tweet in tweets:
#         print('%s,%s' % (tweet.id, tweet.text.encode('UTF-8')))
        
    return [item for sublist in tweets for item in sublist]
        

def chunks_ids(l, n):
    # looping till length l
    for i in range(0, len(l), n): 
        yield l[i:i + n]
 

Twitter Data (Twitter.txt): This corpus contains 992 labeled events in total. 
Each line contains one event with the ids of relevant tweets: event_id, label, tweet_ids. 
For the labels, the value is 1 if the event is a rumor, and is 0 otherwise.
Note that we cannot release the specific content of tweets due to the terms of use of Twitter data.
Users can download the content themselves via Twitter API. 

In [ ]:
content = []
with open('/home/paperspace/sonic/fakeNews/data/rumdect/Twitter.txt') as f:
    content = f.readlines()

tweeter_stories = []
for story in tqdm(content):
    story = story.replace('\t', ' ').split(' ')
#     print(len())
    ids = story[2:-1]
    tweets = get_tweet_list(api, ids)
    for tweet in tweets:
        tweeter_stories.append({ 'event_id' : story[0], 
                                'label' : story[1].split(':')[1], 
                                'tweet_id' : tweet.id, 
                                'tweet_text' : tweet.text.encode('UTF-8')})
#     break

df = pd.DataFrame(tweeter_stories)
df.to_csv('../data/RumerTwitter.csv', sep='\t')
df['label'].value_counts().plot(kind='bar', alpha=.5)
df.tail()

In [9]:
%%time


from nltk.tokenize import TweetTokenizer
import multiprocessing
import numpy as np
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

def sentence_tokenizer(raw_text):
#     return u' '.join([tknzr.tokenize(raw_text)])
    return u' '.join([sent.lower().strip() for sent in tknzr.tokenize(raw_text)])

def _apply_df(args):
    df, func, kwargs = args
    print(df.shape)
    df['text'] = df['tweet_text'].apply(func, **kwargs)
    return df#df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    workers = kwargs.pop('workers')
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs) for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))

num_cores = multiprocessing.cpu_count()
print(num_cores) 

df = pd.read_csv('../data/RumerTwitter.csv', sep='\t')
df = apply_by_multiprocessing(df, sentence_tokenizer,  workers=num_cores)
df.to_csv('../data/RumerTwitter.csv', sep='\t')   

8
(67739, 5)
(67739, 5)
(67739, 5)
(67739, 5)
(67739, 5)
(67739, 5)
(67739, 5)
(67739, 5)
CPU times: user 8.17 s, sys: 972 ms, total: 9.14 s
Wall time: 18.1 s


In [10]:
print(df['label'].value_counts())
len(df['tweet_id'].unique())
df.tail()

0    396731
1    145181
Name: label, dtype: int64


,Unnamed: 0,event_id,label,tweet_id,tweet_text,text
541907,541907,eid:TM1656,0,17868961827,b'RT @Eydeycallme_SK: RT @ToBe757: Smh @ jamar...,b'rt : rt : smh @ jamarcus russell 4 being arr...
541908,541908,eid:TM1656,0,17869549120,b'RT @Eydeycallme_SK:Smh @jamarcus russell 4 b...,b'rt : smh russell 4 being arrested for posses...
541909,541909,eid:TM1656,0,17868922195,b'RT @ToBe757: Smh @ jamarcus russell 4 being ...,b'rt : smh @ jamarcus russell 4 being arrested...
541910,541910,eid:TM1656,0,17844350747,b'http://whatisthetrend.net/jamarcus-russell-%...,b'http :/ / whatisthetrend.net/jamarcus-russel...
541911,541911,eid:TM1656,0,17838159964,b'RT @followadi: jamarcus russell arrested for...,b'rt : jamarcus russell arrested for possessio...


In [ ]:

# def get_tweet_id(line):
#     '''
#     Extracts and returns tweet ID from a line in the input.
#     '''
#     (tagid,_timestamp,_sandyflag) = line.split('\t')
#     (_tag, _search, tweet_id) = tagid.split(':')
#     return tweet_id

# def get_tweets_single(twapi, idfilepath):
#     '''
#     Fetches content for tweet IDs in a file one at a time,
#     which means a ton of HTTPS requests, so NOT recommended.

#     `twapi`: Initialized, authorized API object from Tweepy
#     `idfilepath`: Path to file containing IDs
#     '''
#     # process IDs from the file
#     with open(idfilepath, 'rb') as idfile:
#         for line in idfile:
#             tweet_id = get_tweet_id(line)
#             Logger.debug('get_tweets_single: fetching tweet for ID %s', tweet_id)
#             try:
#                 tweet = twapi.get_status(tweet_id)
#                 print('%s,%s' % (tweet_id, tweet.text.encode('UTF-8')))
#             except tweepy.TweepError as te:
#                 Logger.warn('get_tweets_single: failed to get tweet ID %s: %s', tweet_id, te.message)
#                 # traceback.print_exc(file=sys.stderr)
#         # for
#     # with

# def get_tweet_list(twapi, idlist):
#     '''
#     Invokes bulk lookup method.
#     Raises an exception if rate limit is exceeded.
#     '''
#     # fetch as little metadata as possible
#     tweets = twapi.statuses_lookup(id_=idlist, include_entities=False, trim_user=True)
#     if len(idlist) != len(tweets):
#         Logger.warn('get_tweet_list: unexpected response size %d, expected %d', len(tweets), len(idlist))
#     for tweet in tweets:
#         print('%s,%s' % (tweet.id, tweet.text.encode('UTF-8')))

# def get_tweets_bulk(twapi, idfilepath):
#     '''
#     Fetches content for tweet IDs in a file using bulk request method,
#     which vastly reduces number of HTTPS requests compared to above;
#     however, it does not warn about IDs that yield no tweet.

#     `twapi`: Initialized, authorized API object from Tweepy
#     `idfilepath`: Path to file containing IDs
#     '''    
#     # process IDs from the file
#     tweet_ids = list()
#     with open(idfilepath, 'rb') as idfile:
#         for line in idfile:
#             tweet_id = get_tweet_id(line)
#             Logger.debug('Enqueing tweet ID %s', tweet_id)
#             tweet_ids.append(tweet_id)
#             # API limits batch size
#             if len(tweet_ids) == batch_size:
#                 Logger.debug('get_tweets_bulk: fetching batch of size %d', batch_size)
#                 get_tweet_list(twapi, tweet_ids)
#                 tweet_ids = list()
#     # process remainder
#     if len(tweet_ids) > 0:
#         Logger.debug('get_tweets_bulk: fetching last batch of size %d', len(tweet_ids))
#         get_tweet_list(twapi, tweet_ids)

# def usage():
#     print('Usage: get_tweets_by_id.py [options] file')
#     print('    -s (single) makes one HTTPS request per tweet ID')
#     print('    -v (verbose) enables detailed logging')
#     sys.exit()

# def main(args):
#     logging.basicConfig(level=logging.WARN)
#     global Logger
#     Logger = logging.getLogger('get_tweets_by_id')
#     bulk = True
#     try:
#         opts, args = getopt.getopt(args, 'sv')
#     except getopt.GetoptError:
#         usage()
#     for opt, _optarg in opts:
#         if opt in ('-s'):
#             bulk = False
#         elif opt in ('-v'):
#             Logger.setLevel(logging.DEBUG)
#             Logger.debug("main: verbose mode on")
#         else:
#             usage()
#     if len(args) != 1:
#         usage()
#     idfile = args[0]
#     if not os.path.isfile(idfile):
#         print('Not found or not a file: %s' % idfile, file=sys.stderr)
#         usage()

#     # connect to twitter
#     auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
#     auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)
#     api = tweepy.API(auth)

#     # hydrate tweet IDs
#     if bulk:
#         get_tweets_bulk(api, idfile)
#     else:
#         get_tweets_single(api, idfile)

# if __name__ == '__main__':
#     main(sys.argv[1:])